In [31]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [63]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [33]:
df = pd.read_sql('SELECT * FROM mcrecos', con=database_connection)
#del df['srno']
del df['index']


In [34]:
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [35]:
df.drop_duplicates(inplace = True)
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [36]:
df['Call_Time_normal'] =  pd.to_datetime(df['Call_Time_normal'], format='%Y-%m-%d')
df['Call_Time_normal'] = df['Call_Time_normal'].dt.date
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05
5211,HDIL,83.0,Rajat Bose,2016-01-04
5212,DLF,125.5,Rajat Bose,2016-01-04
5213,Vinati Organics,555.0,Karvy,2016-01-01


In [37]:
start_date = '2017-01-01'
end_date = '2017-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [38]:
mask = (df['Call_Time_normal'] > date_object1) & (df['Call_Time_normal'] <= date_object2)

# 2017 data

In [39]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [40]:
df1

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Wipro,350.0,ICICI Direct,2017-12-29
1,Petronet LNG,285.0,Kotak Securities,2017-12-28
2,Dishman Carbogen Amcis,435.0,HDFC Securities,2017-12-26
3,Simplex Infrastructure,750.0,ICICI Direct,2017-12-26
4,Greenply Industries,425.0,ICICI Direct,2017-12-26
...,...,...,...,...
1842,GSFC,117.0,ICICI Direct,2017-01-10
1843,Hindalco Industries,176.0,Yogesh Mehta,2017-01-10
1844,Solar Industries,901.0,KR Choksey,2017-01-03
1845,Indian Bank,275.0,ICICI Direct,2017-01-03


In [41]:
print("No of Brokers in 2016 :",df1['Analyst_or_Cmp'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 71


## CLEANING

In [42]:
dfsymbols = pd.read_sql("SELECT SYMBOL,recoName FROM symbolsmappedmc", con=database_connection)
dfsymbols

,SYMBOL,recoName
0,SIYSIL,siyaramsilkmills
1,SIYSIL,siyaramsilk
2,JKLAKSHMI,jklakshmicement
3,JKLAKSHMI,jklakshmicements
4,JKLAKSHMI,jklakshmicement
...,...,...
1169,None,uplondeclines
1170,None,jetairwaysondips
1171,None,jenburktpharma
1172,None,samkrgpistons


In [43]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'Cmp_Name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    appenddf = dfsymbols.loc[dfsymbols['recoName']==sym]
    del appenddf['recoName']
    df3 = df3.append(appenddf,ignore_index=True)

In [44]:
len(df1)

1847

In [45]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1939 entries, 0 to 1938
Data columns (total 1 columns):
SYMBOL    1391 non-null object
dtypes: object(1)
memory usage: 15.3+ KB


In [46]:
df3["SYMBOL"].fillna(method='ffill', inplace = True) 

In [47]:
df3=df3.replace("None",np.nan)

In [48]:
df3

,SYMBOL
0,WIPRO
1,PETRONET
2,DCAL
3,SIMPLEXINF
4,SIMPLEXINF
...,...
1934,PVR
1935,HINDALCO
1936,HINDALCO
1937,HINDALCO


## SYMBOLS ADDED TO TABLE

In [49]:
df4 =pd.concat([df1,df3],axis=1)

In [50]:
df4 = df4.dropna()

In [51]:
df4

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal,SYMBOL
0,Wipro,350.0,ICICI Direct,2017-12-29,WIPRO
1,Petronet LNG,285.0,Kotak Securities,2017-12-28,PETRONET
2,Dishman Carbogen Amcis,435.0,HDFC Securities,2017-12-26,DCAL
3,Simplex Infrastructure,750.0,ICICI Direct,2017-12-26,SIMPLEXINF
4,Greenply Industries,425.0,ICICI Direct,2017-12-26,SIMPLEXINF
...,...,...,...,...,...
1842,GSFC,117.0,ICICI Direct,2017-01-10,CHENNPETRO
1843,Hindalco Industries,176.0,Yogesh Mehta,2017-01-10,ULTRACEMCO
1844,Solar Industries,901.0,KR Choksey,2017-01-03,DBCORP
1845,Indian Bank,275.0,ICICI Direct,2017-01-03,DBCORP


In [52]:
uniqueBrokers = df4.Analyst_or_Cmp.unique()
uniqueBrokers

array(['ICICI Direct', 'Kotak Securities', 'HDFC Securities',
       'KR Choksey', 'Motilal Oswal', 'Nirmal Bang', 'Mehta Equites',
       'Edelweiss', 'Dalmia Securities', 'Khambatta Securities',
       'Axis Direct', 'JM Financial', 'Centrum', 'GEPL Capital',
       'Cholamandalam Securities', 'Bonanza', 'Joindre Capital', 'Geojit',
       'Joindre Capital Services', 'Dolat Capital', 'Shahina Mukadam',
       'Centrum Research', 'Cholamandalam securities', 'SPA Securities',
       'Sharekhan', 'Reliance Securities', 'Prabhudas Lilladher',
       'Share India', 'Geojit Research', 'KRChoksey', 'LKP Research',
       'SPA Research', 'BP Equities', 'CD Equisearch',
       'Prabhudas Liladhar', 'Mitessh Thakkar', 'IDBI Capital',
       'Bonanza Portfolio', 'East India Securities', 'Way2wealth',
       'Geojit Financial Services', 'CITI', 'Goldman Sachs',
       'Edelweiss Securities', 'MOST', 'Hem Securities', 'Ashwani Gujral',
       'Dynamic Levels', 'Systematix', 'LKP Securities', 'Pra

In [53]:
#uniqueBrokers = uniqueBrokers[:1]

## PERFORMANCE DEDUCTION ALGORITHM

In [54]:
#del brokerPerformance1
brokerPerformance1 = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos


In [55]:
dfprices2017 = pd.read_sql("SELECT * FROM nse2017", con=database_connection)
dfprices2017

,index,Symbol,Date,Closeval
0,0,20MICRONS,2017-01-02,33.85
1,1,3IINFOTECH,2017-01-02,5.60
2,2,3MINDIA,2017-01-02,11056.45
3,3,8KMILES,2017-01-02,674.20
4,4,A2ZINFRA,2017-01-02,42.15
...,...,...,...,...
403386,403393,NIFTYHIGHBETA50,2017-12-29,2396.70
403387,403394,NIFTYLOWVOL50,2017-12-29,10102.21
403388,403395,NIFTY50VALUE20,2017-12-29,4732.00
403389,403396,NIFTYGROWSECT15,2017-12-29,6576.85


In [56]:
dfprices2018 = pd.read_sql("SELECT * FROM nse2017", con=database_connection)
dfprices2018

,index,Symbol,Date,Closeval
0,0,20MICRONS,2017-01-02,33.85
1,1,3IINFOTECH,2017-01-02,5.60
2,2,3MINDIA,2017-01-02,11056.45
3,3,8KMILES,2017-01-02,674.20
4,4,A2ZINFRA,2017-01-02,42.15
...,...,...,...,...
403386,403393,NIFTYHIGHBETA50,2017-12-29,2396.70
403387,403394,NIFTYLOWVOL50,2017-12-29,10102.21
403388,403395,NIFTY50VALUE20,2017-12-29,4732.00
403389,403396,NIFTYGROWSECT15,2017-12-29,6576.85


In [59]:
lost =[]
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['Analyst_or_Cmp'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(sym)

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['Call_Time_normal']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
            print(mydate1)
            
            #Fetching current price from nse bav copy
            #cnx = mysql.connector.connect(user="root", password='yash', host="127.0.0.1", port=3306, database='mainschema')
            #sql_select_Query = "SELECT Closeval FROM "+dbyear1+" WHERE Date = '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            #cursor = cnx.cursor()
            #cursor.execute(sql_select_Query)
            #record = cursor.fetchall()
            record = dfprices2017.loc[dfprices2017['Symbol']== sym]
            record = record.loc[record['Date']== mydate1]
            if len(record) != 0:
                
                x = record['Closeval'].iloc[0]
            
        
                #sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
                #dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
                #del dfprices1['index']
                dfprices1 = dfprices2017[dfprices2017['Date'] >= mydate1]
                del dfprices1['index']
        
                #sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
                #dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
                #del dfprices2['index']
                #dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)
                dfprices2 = dfprices2018[dfprices2018['Date'] <= mydate2]
                del dfprices2['index']
                dfprices1 = pd.concat([dfprices1,dfprices2],ignore_index=True)
                dfprices = dfprices1[dfprices1['Symbol'] == sym]
                #targetp = float(row['target_price'])

                targetp = float(row['Target_Price'])
        
                print("SYMBOL : ",sym)
                dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
                total = total + 1
            
                if not dfhits.empty :
                    print("HIT!")
                    hits = hits + 1
                    #recbuying = float(row['recomended_buying'])
                    maxval = dfprices['Closeval'].max()
                    growthperc = (maxval - x) *100 / x
                    print('Growth : ',growthperc)
                    growthSum = growthSum + growthperc
                    
                
                else:
                    print("DIDN'T HIT!")
                del dfprices
                del dfhits
            else:
                lost.append(sym)
    if total != 0: 
        hitRatio = int(hits*100 / total)
        print("Growthsum",growthSum)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformance1 = brokerPerformance1.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  ICICI Direct
WIPRO
2017-12-29
SYMBOL :  WIPRO
HIT!
Growth :  78.37708830548925
SIMPLEXINF
2017-12-26
SYMBOL : 

C:\Users\dhruv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
C:\Users\dhruv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
C:\Users\dhruv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced 

 SIMPLEXINF
DIDN'T HIT!
SIMPLEXINF
2017-12-26
SYMBOL :  SIMPLEXINF
HIT!
Growth :  2.2020725388601035
GUJGASLTD
2017-12-26
SYMBOL :  GUJGASLTD
DIDN'T HIT!
GUJGASLTD
2017-12-26
SYMBOL :  GUJGASLTD
DIDN'T HIT!
BAJAJELEC
2017-12-26
SYMBOL :  BAJAJELEC
DIDN'T HIT!
SHANKARA
2017-12-06
SYMBOL :  SHANKARA
DIDN'T HIT!
DISHTV
2017-12-06
SYMBOL :  DISHTV
HIT!
Growth :  34.16870415647922
ALKEM
2017-11-28
SYMBOL :  ALKEM
HIT!
Growth :  11.927719990889077
SOMANYCERA
2017-11-28
SYMBOL :  SOMANYCERA
HIT!
Growth :  8.364842068524238
SIYSIL
2017-11-28
SYMBOL :  SIYSIL
HIT!
Growth :  332.71882261812544
RAMCOCEM
2017-11-15
SYMBOL :  RAMCOCEM
HIT!
Growth :  15.466933009916527
CIPLA
2017-11-15
SYMBOL :  CIPLA
HIT!
Growth :  10.300682220163393
DBCORP
2017-11-15
SYMBOL :  DBCORP
DIDN'T HIT!
DBCORP
2017-11-15
SYMBOL :  DBCORP
HIT!
Growth :  8.24486419412658
DBCORP
2017-11-15
SYMBOL :  DBCORP
DIDN'T HIT!
TECHM
2017-11-15
SYMBOL :  TECHM
HIT!
Growth :  3.56459571040177
EQUITAS
2017-11-07
SYMBOL :  EQUITAS
DIDN'T

SYMBOL :  TIIL
DIDN'T HIT!
RAMCOCEM
2017-06-01
SYMBOL :  RAMCOCEM
HIT!
Growth :  12.030179275974248
NTPC
2017-06-01
SYMBOL :  NTPC
HIT!
Growth :  15.997510115157167
BHARATFORG
2017-06-01
SYMBOL :  BHARATFORG
HIT!
Growth :  5.722104739395448
GRASIM
2017-06-01
SYMBOL :  GRASIM
HIT!
Growth :  14.438221911095555
BRIGADE
2017-05-31
SYMBOL :  BRIGADE
DIDN'T HIT!
PNCINFRA
2017-05-31
SYMBOL :  PNCINFRA
HIT!
Growth :  47.65465669612508
BHARATFORG
2017-05-31
SYMBOL :  BHARATFORG
DIDN'T HIT!
CROMPTON
2017-05-31
SYMBOL :  CROMPTON
DIDN'T HIT!
INSECTICID
2017-05-31
SYMBOL :  INSECTICID
HIT!
Growth :  46.18223760092272
CUB
2017-05-29
SYMBOL :  CUB
DIDN'T HIT!
IGL
2017-05-29
SYMBOL :  IGL
HIT!
Growth :  57.29252960382049
AUROPHARMA
2017-05-29
SYMBOL :  AUROPHARMA
HIT!
Growth :  55.32786885245902
JKIL
2017-05-29
SYMBOL :  JKIL
DIDN'T HIT!
JKIL
2017-05-29
SYMBOL :  JKIL
DIDN'T HIT!
TECHNOE
2017-05-29
JAGRAN
2017-05-26
SYMBOL :  JAGRAN
DIDN'T HIT!
JAGRAN
2017-05-26
SYMBOL :  JAGRAN
DIDN'T HIT!
BAJAJELEC

SYMBOL :  JUBILANT
HIT!
Growth :  21.211475642664396
GULFOILLUB
2017-02-07
SYMBOL :  GULFOILLUB
HIT!
Growth :  46.63664307670312
J&KBANK
2017-02-07
SYMBOL :  J&KBANK
DIDN'T HIT!
J&KBANK
2017-02-03
SYMBOL :  J&KBANK
DIDN'T HIT!
LUPIN
2017-02-03
SYMBOL :  LUPIN
HIT!
Growth :  1.8264075067024128
HSIL
2017-02-03
SYMBOL :  HSIL
HIT!
Growth :  86.77220756376428
LAOPALA
2017-02-03
SYMBOL :  LAOPALA
HIT!
Growth :  36.51385390428212
ABBOTINDIA
2017-02-03
SYMBOL :  ABBOTINDIA
HIT!
Growth :  26.234788833214036
ZEEL
2017-02-03
SYMBOL :  ZEEL
DIDN'T HIT!
INDHOTEL
2017-02-03
SYMBOL :  INDHOTEL
DIDN'T HIT!
BIRLACORPN
2017-02-02
SYMBOL :  BIRLACORPN
DIDN'T HIT!
PVR
2017-02-02
SYMBOL :  PVR
DIDN'T HIT!
BRIGADE
2017-02-02
SYMBOL :  BRIGADE
DIDN'T HIT!
SONATSOFTW
2017-02-02
SYMBOL :  SONATSOFTW
DIDN'T HIT!
VINATIORGA
2017-02-02
SYMBOL :  VINATIORGA
HIT!
Growth :  46.43486512132302
GUJGASLTD
2017-02-01
SYMBOL :  GUJGASLTD
HIT!
Growth :  63.35987800745508
JKLAKSHMI
2017-02-01
SYMBOL :  JKLAKSHMI
DIDN'T HIT

...

SYMBOL :  APCOTEXIND
HIT!
Growth :  34.415497661990656
JETAIRWAYS
2017-06-06
SYMBOL :  JETAIRWAYS
HIT!
Growth :  67.32703137570395
SADBHAV
2017-06-06
SYMBOL :  SADBHAV
DIDN'T HIT!
ZENSARTECH
2017-06-05
SYMBOL :  ZENSARTECH
HIT!
Growth :  9.24369747899159
MANPASAND
2017-06-05
SYMBOL :  MANPASAND
HIT!
Growth :  22.179816054557968
MANPASAND
2017-06-05
SYMBOL :  MANPASAND
DIDN'T HIT!
MANPASAND
2017-06-05
SYMBOL :  MANPASAND
HIT!
Growth :  22.179816054557968
JAGRAN
2017-05-26
SYMBOL :  JAGRAN
HIT!
Growth :  10.761081683858384
INDIACEM
2017-05-26
SYMBOL :  INDIACEM
DIDN'T HIT!
NRBBEARING
2017-05-22
SYMBOL :  NRBBEARING
DIDN'T HIT!
GAIL
2017-05-22
SYMBOL :  GAIL
DIDN'T HIT!
BRIGADE
2017-05-22
SYMBOL :  BRIGADE
HIT!
Growth :  29.36540016168149
BRIGADE
2017-05-22
SYMBOL :  BRIGADE
DIDN'T HIT!
ENIL
2017-05-22
SYMBOL :  ENIL
DIDN'T HIT!
GAIL
2017-05-22
SYMBOL :  GAIL
DIDN'T HIT!
NCC
2017-05-22
SYMBOL :  NCC
DIDN'T HIT!
DHANUKA
2017-05-22
SYMBOL :  DHANUKA
HIT!
Growth :  15.727507397846752
GAIL
20

SYMBOL :  KALPATPOWR
DIDN'T HIT!
INDUSINDBK
2017-09-08
SYMBOL :  INDUSINDBK
DIDN'T HIT!
VEDL
2017-09-04
SYMBOL :  VEDL
DIDN'T HIT!
JAGRAN
2017-08-04
SYMBOL :  JAGRAN
DIDN'T HIT!
Growthsum 220.5625600074336
Broker :  JM Financial HitRatio :  38 Avg. Growth :  7 Total Recos. :  31
brokerName :  Centrum
L&TFH
2017-12-05
SYMBOL :  L&TFH
DIDN'T HIT!
GDL
2017-11-27
SYMBOL :  GDL
DIDN'T HIT!
DLF
2017-11-22
SYMBOL :  DLF
DIDN'T HIT!
SMLISUZU
2017-11-22
SYMBOL :  SMLISUZU
HIT!
Growth :  51.277937596813466
NTPC
2017-11-16
SYMBOL :  NTPC
DIDN'T HIT!
MAHSEAMLES
2017-11-16
SYMBOL :  MAHSEAMLES
HIT!
Growth :  13.150624378384352
MAHSEAMLES
2017-11-16
SYMBOL :  MAHSEAMLES
DIDN'T HIT!
SHREECEM
2017-11-16
SYMBOL :  SHREECEM
HIT!
Growth :  12.837385679232657
MARUTI
2017-11-02
SYMBOL :  MARUTI
HIT!
Growth :  19.155583651438167
MARUTI
2017-11-02
SYMBOL :  MARUTI
HIT!
Growth :  19.155583651438167
FEDERALBNK
2017-11-02
SYMBOL :  FEDERALBNK
DIDN'T HIT!
BAJFINANCE
2017-10-16
SYMBOL :  BAJFINANCE
HIT!
Growth : 

SYMBOL :  IOC
DIDN'T HIT!
DABUR
2017-11-03
SYMBOL :  DABUR
DIDN'T HIT!
SYMPHONY
2017-11-03
SYMBOL :  SYMPHONY
HIT!
Growth :  15.488280002554767
MHRIL
2017-11-03
SYMBOL :  MHRIL
HIT!
Growth :  82.0248222005299
VEDL
2017-11-03
SYMBOL :  VEDL
DIDN'T HIT!
ERIS
2017-10-26
SYMBOL :  ERIS
DIDN'T HIT!
ALLCARGO
2017-10-25
SYMBOL :  ALLCARGO
DIDN'T HIT!
BLUEDART
2017-10-23
SYMBOL :  BLUEDART
HIT!
Growth :  28.00535475234271
HAVELLS
2017-10-23
SYMBOL :  HAVELLS
DIDN'T HIT!
HAVELLS
2017-10-23
SYMBOL :  HAVELLS
DIDN'T HIT!
HAVELLS
2017-10-23
SYMBOL :  HAVELLS
HIT!
Growth :  3.8784744667097604
CANFINHOME
2017-10-09
SYMBOL :  CANFINHOME
DIDN'T HIT!
GAIL
2017-10-06
SYMBOL :  GAIL
HIT!
Growth :  15.948751932847358
DIVISLAB
2017-10-04
SYMBOL :  DIVISLAB
DIDN'T HIT!
AKZOINDIA
2017-09-28
SYMBOL :  AKZOINDIA
HIT!
Growth :  9.545833445373631
PEL
2017-09-28
SYMBOL :  PEL
HIT!
Growth :  18.94997176296468
CGCL
2017-09-27
SYMBOL :  CGCL
DIDN'T HIT!
RKFORGE
2017-09-26
SYMBOL :  RKFORGE
DIDN'T HIT!
MOTHERSUMI
201

SYMBOL :  PEL
HIT!
Growth :  18.94997176296468
OBEROIRLTY
2017-09-22
SYMBOL :  OBEROIRLTY
DIDN'T HIT!
Growthsum 18.94997176296468
Broker :  Prabhudas Liladhar HitRatio :  50 Avg. Growth :  9 Total Recos. :  2
brokerName :  Mitessh Thakkar
RAMASTEEL
2017-09-06
SYMBOL :  RAMASTEEL
HIT!
Growth :  38.36234687298517
JAGRAN
2017-05-29
SYMBOL :  JAGRAN
DIDN'T HIT!
BLUEDART
2017-05-10
SYMBOL :  BLUEDART
HIT!
Growth :  12.321131092221618
TCI
2017-04-20
SYMBOL :  TCI
DIDN'T HIT!
INDIGO
2017-03-09
SYMBOL :  INDIGO
HIT!
Growth :  50.05427640975834
HDFCBANK
2017-03-08
SYMBOL :  HDFCBANK
HIT!
Growth :  35.36129866398505
DHFL
2017-03-01
SYMBOL :  DHFL
HIT!
Growth :  100.34612490594432
DISHTV
2017-01-24
SYMBOL :  DISHTV
DIDN'T HIT!
Growthsum 236.4451779448945
Broker :  Mitessh Thakkar HitRatio :  62 Avg. Growth :  29 Total Recos. :  8
brokerName :  IDBI Capital
DBL
2017-08-28
SYMBOL :  DBL
HIT!
Growth :  65.77510040160644
DELTACORP
2017-08-23
SYMBOL :  DELTACORP
HIT!
Growth :  75.66529492455417
MARUTI

brokerName :  Geojit BNP Paribas
SPICEJET
2017-02-17
RAMCOCEM
2017-02-10
SYMBOL :  RAMCOCEM
HIT!
Growth :  17.486933797909415
BANKBARODA
2017-02-10
SYMBOL :  BANKBARODA
DIDN'T HIT!
CARERATING
2017-02-03
SYMBOL :  CARERATING
HIT!
Growth :  20.209442669506572
INTELLECT
2017-01-31
SYMBOL :  INTELLECT
DIDN'T HIT!
DIVISLAB
2017-01-30
SYMBOL :  DIVISLAB
DIDN'T HIT!
CADILAHC
2017-01-27
SYMBOL :  CADILAHC
DIDN'T HIT!
CADILAHC
2017-01-27
SYMBOL :  CADILAHC
HIT!
Growth :  51.51055317974893
EICHERMOT
2017-01-27
SYMBOL :  EICHERMOT
HIT!
Growth :  39.94140354687395
IOC
2017-01-25
SYMBOL :  IOC
DIDN'T HIT!
VGUARD
2017-01-25
SYMBOL :  VGUARD
HIT!
Growth :  29.34023285899094
VGUARD
2017-01-25
SYMBOL :  VGUARD
DIDN'T HIT!
KTKBANK
2017-01-20
SYMBOL :  KTKBANK
DIDN'T HIT!
HERITGFOOD
2017-01-20
SYMBOL :  HERITGFOOD
HIT!
Growth :  69.02854006586168
ASIANPAINT
2017-01-10
SYMBOL :  ASIANPAINT
DIDN'T HIT!
Growthsum 227.51710611889146
Broker :  Geojit BNP Paribas HitRatio :  42 Avg. Growth :  16 Total Recos. :

In [60]:
len(lost)

42

In [61]:
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos
0,ICICI Direct,48,17,321
1,Kotak Securities,50,18,14
2,HDFC Securities,49,21,159
3,KR Choksey,48,35,80
4,Motilal Oswal,50,24,303
...,...,...,...,...
63,Religare,80,19,5
64,Anand Rathi,75,27,4
65,ICICI Securities,0,0,1
66,AUM Capital,100,92,1


In [64]:
brokerPerformance1.to_sql(con=database_connection, name='mcbrokerperformance2017', if_exists='replace')